# Variables

## Import des librairies

In [ ]:
import os
import sys
from rdflib import Graph, Namespace, URIRef, Literal, BNode, XSD

## Définition des variables globales

In [ ]:
# Fichiers existants
addr_ont_file_name = "address_ont.ttl"
ev_ont_file_name = "events_ont.ttl"
ont_file_name = "merge_addr_ont.ttl" # Fusion des deux fichiers précédents
ruleset_file_name = "rules_sameAs.pie"

# Fichiers créés durant le processus
export_file_name = "addresses-temp.ttl"
out_file_name = "addresses.ttl"
local_config_file_name = "config_repo.ttl"

# Dossiers existants
data_folder_name = "data"
mapping_folder_name = "mappings"

# Dossiers créés durant le traitement
tmp_folder_name = "tmp_files"

project_name = "addresses_from_factoids_same_as"
ontology_named_graph_name = "ontology"

graphdb_url = "http://localhost:7200"
ontorefine_url = "http://localhost:7333"
local_uri = "http://rdf.geohistoricaldata.org/id/address/"

# Commande pour lancer `Ontorefine`, dépend de l'OS et d'où il se situe
# ontorefine_cmd = "ontorefine-cli" # Nom sans chemin
ontorefine_cmd = "/opt/ontotext-refine/lib/app/bin/ontorefine-cli" #Ubuntu
# ontorefine_cmd = "/Applications/Ontotext\ Refine.app/Contents/app/bin/ontorefine-cli" #MacOS

addr_graph_name = "addresses_from_factoids"

py_code_folder_path = "../code"

## Traitement des variables globales

* Obtention des chemins absolus des fichiers à partir des chemins relatifs donnés dans la section précédente
* Création d'un dossier temporaire s'il n'existe pas encore pour stocker des fichiers à vocation d'être supprimés

In [ ]:
tmp_folder = os.path.abspath(tmp_folder_name)
mapping_folder = os.path.abspath(mapping_folder_name)
data_folder = os.path.abspath(data_folder_name)

python_code_folder = os.path.abspath(py_code_folder_path)

local_config_file = os.path.join(tmp_folder, local_config_file_name)
addr_ont_file = os.path.abspath(addr_ont_file_name)
ev_ont_file = os.path.abspath(ev_ont_file_name)
ont_file = os.path.abspath(ont_file_name)
ruleset_file = os.path.abspath(ruleset_file_name)

## Import des modules situés dans le dossier `code`

In [ ]:
# Appel du dossier `code` comprend les codes python
sys.path.insert(1, python_code_folder)

import filemanagement as fm
import wikidata as wd
import ontorefine as otr
import graphdb as gd
import graphrdf as gr
import strprocessing as sp

## Création de dossiers s'ils n'existent pas

In [ ]:
fm.create_folder_if_not_exists(tmp_folder)

## Gestion du répertoire local

### Création du répertoire local dans GraphDB
Pour que la création marche, il faut que GraphDB soit lancé et donc que l'URI donné par `graphdb_url` fonctionne. Si le répertoire existe déjà, rien n'est fait

In [ ]:
gd.remove_repository(graphdb_url, project_name)
gd.create_config_local_repository_file(local_config_file, project_name, ruleset_file=ruleset_file)
# gd.create_config_local_repository_file(local_config_file, project_name)
gd.create_repository_from_config_file(graphdb_url, local_config_file)

### Vidage du répertoire local
Le répertoire dont l'id est `project_name` pour y stocker les données récupérées, cela est utile si le répertoire existait déjà.

In [ ]:
gd.clear_repository(graphdb_url, project_name)

## Import des ontologies

In [ ]:
# Import de l'ontologie
# gd.import_ttl_file_in_graphdb(graphdb_url, project_name, addr_ont_file)
# gd.import_ttl_file_in_graphdb(graphdb_url, project_name, ev_ont_file)
gd.import_ttl_file_in_graphdb(graphdb_url, project_name, ont_file, ontology_named_graph_name)

## Définition de variables liées aux sources et aux faits

### Graphe de faits, accumulation des données des sources

In [ ]:
# Définition du graphe nommé
facts_graph_name = "facts"
facts_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, project_name, facts_graph_name))

### Voies de Paris via Wikidata

In [ ]:
# `wdpt` pour "wikidata paris thoroughfares"

# Fichier CSV pour stocker le résultat de la requête de la sélection
wdpt_csv_file_name = "wd_paris_thoroughfares.csv"
wdpt_csv_file = os.path.join(tmp_folder, wdpt_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
wdpt_mapping_file_name = "mapping_voies_wikidata.json"
wdpt_mapping_file = os.path.join(mapping_folder, wdpt_mapping_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
wdpt_kg_file_name = "wd_paris_thoroughfares.ttl"
wdpt_kg_file = os.path.join(tmp_folder, wdpt_kg_file_name)

# Définition des graphes nommés
wdpt_graph_name = "wikidata"
wdpt_ids_graph_name = "wikidata_ids"
wdpt_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, project_name, wdpt_graph_name))
wdpt_ids_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, project_name, wdpt_ids_graph_name))

### Nomenclature des voies de la ville de Paris

Les données de la ville de Paris sont composées de deux jeux :
* [dénominations des emprises des voies actuelles](https://opendata.paris.fr/explore/dataset/denominations-emprises-voies-actuelles)
* [dénominations caduques des voies](https://opendata.paris.fr/explore/dataset/denominations-des-voies-caduques)

Les voies actuelles ont une emprise géométrique contrairement aux anciennes voies.

In [ ]:
# `vpt` pour "ville paris thoroughfares"

# Fichier CSV pour stocker le résultat de la requête de la sélection
vpta_csv_file_name = "denominations-emprises-voies-actuelles.csv"
vpta_csv_file = os.path.join(data_folder, vpta_csv_file_name)
vptc_csv_file_name = "denominations-des-voies-caduques.csv"
vptc_csv_file = os.path.join(data_folder, vptc_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
vpta_mapping_file_name = "mapping_voies_paris_actuelles.json"
vpta_mapping_file = os.path.join(mapping_folder, vpta_mapping_file_name)
vptc_mapping_file_name = "mapping_voies_paris_caduques.json"
vptc_mapping_file = os.path.join(mapping_folder, vptc_mapping_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
vpta_kg_file_name = "voies_paris_actuelles.ttl"
vpta_kg_file = os.path.join(tmp_folder, vpta_kg_file_name)
vptc_kg_file_name = "voies_paris_caduques.ttl"
vptc_kg_file = os.path.join(tmp_folder, vptc_kg_file_name)

# Définition des graphes nommés
vpt_graph_name = "ville_de_paris"
vpt_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, project_name, vpt_graph_name))

### Base Adresse Nationale (BAN)

Données de la [Base Adresse Nationale (BAN)](https://adresse.data.gouv.fr/base-adresse-nationale), accessible [ici](https://adresse.data.gouv.fr/data/ban/adresses/latest/csv)

In [ ]:
# `bpa` pour "BAN paris addresses"

# Fichier CSV pour stocker le résultat de la requête de la sélection
bpa_csv_file_name = "ban_adresses.csv"
bpa_csv_file = os.path.join(data_folder, bpa_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
bpa_mapping_file_name = "mapping_ban_adresses.json"
bpa_mapping_file = os.path.join(mapping_folder, bpa_mapping_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
bpa_kg_file_name = "ban_adresses.ttl"
bpa_kg_file = os.path.join(tmp_folder, bpa_kg_file_name)

# Définition des graphes nommés
bpa_graph_name = "ban"
bpa_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, project_name, bpa_graph_name))

## Fonctions pour des traitements de construction des graphes de factoïdes

### Ajout de labels normalisés

* Étant donné les variations de labels qui sont similaires, la normalisation consiste ici à :
    * la mise en minuscule des caractères
    * suppression des signes diacritiques (accents, cédille...)
    * suppression de termes "inutiles" : déterminants, prépositions...
* Ces labels normalisés sont liés aux repères via `skos:hiddenLabel`

In [ ]:
def add_normalized_label_for_landmarks(graphdb_url, project_name, source_graph_uri:URIRef):
    """
    Ajout de labels normalisés pour les repères via `skos:hiddenLabel` dans le graphe nommé temporaire `tmp_graph_uri`
    """

    label_var = "?label"
    norm_label_var = "?normLabel"
    norm_label_function = sp.get_lower_simplified_french_street_name_function(label_var)

    prefixes = """
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
    PREFIX atype: <http://rdf.geohistoricaldata.org/id/codes/address/attributeType/>
    """

    query = prefixes + f"""
    INSERT {{
        GRAPH ?g {{
            ?landmark skos:hiddenLabel {norm_label_var}.
        }}
    }}
    WHERE {{
        BIND({source_graph_uri.n3()} AS ?g)
        GRAPH ?g {{
            ?landmark a addr:Landmark; addr:hasAttribute [a addr:Attribute; addr:isAttributeType atype:NameAttribute; addr:version [a addr:AttributeVersion; addr:versionValue {label_var}]].
            BIND({norm_label_function} AS {norm_label_var})
        }}
    }}
    """

    gd.update_query(query, graphdb_url, project_name)

### Suppression des instants temporels sans timeStamp

Il existe des resources de classe `addr:TimeInstant` qui ont un calendrier et une granularité dans avoir de time stamp. Il faut supprimer ces ressources qui n'ont aucune utilité.

In [ ]:
def remove_time_instant_without_timestamp(graphdb_url, project_name, source_graph_uri:URIRef):
    query = f"""
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
    DELETE {{
        ?timeInstant ?p ?o.
        ?s ?p ?timeInstant.
    }}WHERE {{
        GRAPH {source_graph_uri.n3()} {{
            ?timeInstant a addr:TimeInstant.
            MINUS {{?timeInstant addr:timeStamp ?timeStamp}}
            {{?timeInstant ?p ?o}}UNION{{?s ?p ?timeInstant}}
        }}
    }}
    """

    gd.update_query(query, graphdb_url, project_name)

### Ajout d'un lien entre les repères et les sources

Un factoïde est la représentation d'une information, d'un fait dans une source. Ici, on crée des repères qui ont une identité définie par la source. Ces derniers doivent avoir un lien avec la source afin d'attester la provenance de son existence.

Pour cela, on sélectionne l'ensemble des repères (`?landmark`) situés dans le graphe nommé défini par `source_graph_uri` afin de créer le triplet `<?landmark rico:isOrWasDescribedBy ?sourceUri>` où `?sourceUri` est l'URI décrivant la source.

In [ ]:
def add_source_resources_links(graphdb_url, project_name, source_resource_uri:URIRef, source_graph_uri:URIRef):
    query = f"""
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
    PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
    PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>

    INSERT {{
        GRAPH ?g {{
            ?elem rico:isOrWasDescribedBy {source_resource_uri.n3()}.
        }}
    }}WHERE {{
        {{?type rdfs:subClassOf* addr:Landmark}} UNION
        {{?type rdfs:subClassOf* addr:LandmarkRelation}} UNION
        {{?type rdfs:subClassOf* addr:AttributeVersion}} UNION
        {{?type rdfs:subClassOf* addr:Address}} UNION
        {{?type rdfs:subClassOf* addr:Change}} UNION
        {{?type rdfs:subClassOf* addr:Event}} UNION
        {{?type rdfs:subClassOf* addr:TemporalEntity}}

        BIND({source_graph_uri.n3()} AS ?g)
        GRAPH ?g {{
            ?elem a ?type.
        }}
    }}
    """

    gd.update_query(query, graphdb_url, project_name)

### Conversion du fichier brut vers le graphe dans GraphDB

À partir un fichier brut (un fichier tabulaire comme un CSV), la fonction le convertit en graphe de connaissances dans un fichier `ttl` (ici `kg_file`). La manière de convertir le fichier est défini par le fichier `ontorefine_mapping_file`, la conversion se fait par Ontotext Refine. Par la suite, le fichier `ttl` est importé dans le répertoire dont le nom est `project_name`, plus précisément dans le graphe nommé `graph_name`.

In [ ]:
def from_raw_to_data_to_graphdb(graphdb_url, ontorefine_url, ontorefine_cmd, project_name, graph_name, csv_file, ontorefine_mapping_file, kg_file):
    # Si ça ne marche pas ici, c'est sûrement qu'Ontotext Refine n'est pas lancé
    otr.get_export_file_from_ontorefine(csv_file, ontorefine_mapping_file, kg_file, ontorefine_cmd, ontorefine_url, project_name)

    # Importer le fichier `kg_file` qui a été créé lors de la ligne précédente dans le répertoire `project_name`, dans le graphe nommé `graph_name` 
    gd.import_ttl_file_in_graphdb(graphdb_url, project_name, kg_file, graph_name)

## Construction des factoïdes pour chaque source

### Nomenclature des voies de la ville de Paris

#### Création des données liées à la source

In [ ]:
def create_source_ville_paris(graphdb_url, project_name, source_resource_uri:URIRef, facts_graph_uri:URIRef):
    query = f"""
        PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
        PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
        PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>

        INSERT DATA {{
            GRAPH {facts_graph_uri.n3()} {{
                {source_resource_uri.n3()} a rico:Record;
                    rdfs:label "dénomination des voies de Paris (actuelles et caduques)"@fr;
                    rico:hasPublisher facts:DirTopoDocFoncVP .
                facts:DirTopoDocFoncVP a rico:CorporateBody;
                    rdfs:label "Département de la Topographie et de la Documentation Foncière de la Ville de Paris"@fr.    
            }}
        }}
        """
    
    gd.update_query(query, graphdb_url, project_name)

#### Définition d'un processus de création des données de la Ville de Paris

In [ ]:
def create_factoid_process_ville_paris(graphdb_url, project_name,
                                       ontorefine_url, ontorefine_cmd,
                                        vpt_graph_name, facts_graph_name, 
                                        vpta_csv_file, vptc_csv_file,
                                        vpta_mapping_file, vptc_mapping_file,
                                        vpta_kg_file, vptc_kg_file):
    """
    Fonction pour faire l'ensemble des processus relatifs à la création des factoïdes pour les données de la dénomination des voies de la ville de Paris
    """

    # Récupération des URI des graphes nommés
    vpt_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, project_name, vpt_graph_name))
    facts_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, project_name, facts_graph_name))

    # A partir des fichiers csv décrivant les voies de la ville de Paris, convertir en un graphe de connaissance selon le mapping défini
    # Puis import du graphe dans le répertoire dont le nom est `project_name` et dans le graphe nommé donné par `graph_name`
    from_raw_to_data_to_graphdb(graphdb_url, ontorefine_url, ontorefine_cmd, project_name, vpt_graph_name, vpta_csv_file, vpta_mapping_file, vpta_kg_file)
    from_raw_to_data_to_graphdb(graphdb_url, ontorefine_url, ontorefine_cmd, project_name, vpt_graph_name, vptc_csv_file, vptc_mapping_file, vptc_kg_file)

    # Suppression des instants qui n'ont aucun timeStamp (instants sans date)
    remove_time_instant_without_timestamp(graphdb_url, project_name, vpt_graph_uri)
    
    # L'URI ci-dessous définit la source liée à la ville de Paris
    vdp_source_uri = URIRef("http://rdf.geohistoricaldata.org/id/address/facts/Source_VDP")
    create_source_ville_paris(graphdb_url, project_name, vdp_source_uri, facts_graph_uri)

    # Ajout de labels normalisés
    add_normalized_label_for_landmarks(graphdb_url, project_name, vpt_graph_uri)

    # Ajout de liens entre les ressources de type repère et la source
    add_source_resources_links(graphdb_url, project_name, vdp_source_uri, vpt_graph_uri)

### Base Adresse Nationale (BAN)

#### Création des données liées à la source

In [ ]:
def create_source_ban(graphdb_url, project_name, source_resource_uri:URIRef, facts_graph_uri:URIRef):
    query = f"""
        PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
        PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
        PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>

        INSERT DATA {{
            GRAPH {facts_graph_uri.n3()} {{
                {source_resource_uri.n3()} a rico:Record;
                    rdfs:label "Base Nationale Adresse"@fr;
                    rico:hasPublisher facts:DINUM_ANCT_IGN .
                facts:IGN_DINUM_ANCT a rico:CorporateBody;
                    rdfs:label "DINUM / ANCT / IGN"@fr.
            }}
        }}
        """
    
    gd.update_query(query, graphdb_url, project_name)

#### Nettoyage du graphe

Détection de repères décrits via plusieurs ressources et création d'une unique ressource pour chaque.

In [ ]:
# Détection des communes et arrondissements dupliqués et création d'une source centrale (?newLandmark) selon le code INSEE.
def clean_ban_graph(graphdb_url, project_name, source_graph_uri):
    prefixes = """
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX geofla: <http://data.ign.fr/def/geofla#>
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
    PREFIX ban: <http://rdf.geohistoricaldata.org/id/address/sources/ban/>
    """

    label_var = "?label"
    norm_label_var = "?normLabel"
    norm_label_function = sp.get_lower_simplified_french_street_name_function(label_var)

    query1 = prefixes + f"""
    INSERT {{
        GRAPH ?g {{
            ?landmark skos:hiddenLabel {norm_label_var}.
        }}
    }}
    WHERE {{
        BIND({source_graph_uri.n3()} AS ?g)
        GRAPH ?g {{
            ?landmark a addr:Landmark; rdfs:label {label_var}.
            BIND({norm_label_function} AS {norm_label_var})
        }}
    }}
    """

    query2 = prefixes + f"""
    INSERT {{
        GRAPH ?g {{
            ?landmark skos:exactMatch ?tmpLandmark.
        }}
    }}
    WHERE
    {{
        BIND({source_graph_uri.n3()} AS ?g)
        {{
            SELECT DISTINCT ?insee {{
                GRAPH ?g {{
                    ?tmpLandmark a addr:Landmark; geofla:numInsee ?insee.
                }}
            }}
        }}
        BIND(URI(CONCAT(STR(URI(ban:)), "LM_", STRUUID())) AS ?landmark)

        GRAPH ?g {{
            ?tmpLandmark a addr:Landmark; addr:isLandmarkType ?landmarkType; geofla:numInsee ?insee.
        }}
    }}
    """

    # Détection des doublons sur les codes postaux

    query3 = prefixes + f"""
    INSERT {{
        GRAPH ?g {{
            ?landmark skos:exactMatch ?tmpLandmark.
        }}
    }}
    WHERE
    {{
        BIND({source_graph_uri.n3()} AS ?g)
        {{
            SELECT DISTINCT ?postalCode {{
                GRAPH ?g {{
                    ?tmpLandmark a addr:Landmark; addr:isLandmarkType ltype:PostalCode; rdfs:label ?postalCode.
                }}
            }}
        }}
        BIND(URI(CONCAT(STR(URI(ban:)), "LM_", STRUUID())) AS ?landmark)

        GRAPH ?g {{
            ?tmpLandmark a addr:Landmark; addr:isLandmarkType ltype:PostalCode; rdfs:label ?postalCode.
        }}
    }}
    """

    # Détection des doublons sur les voies (même nom et appartient à la même commune ou au même arrondissement)

    query4 = prefixes + f"""
    INSERT {{
        GRAPH ?g {{
            ?landmark skos:exactMatch ?tmpLandmark.
        }}
    }}
    WHERE
    {{
        BIND({source_graph_uri.n3()} AS ?g)
        {{
            SELECT DISTINCT ?label ?district WHERE {{
                GRAPH ?g {{
                    ?tmpLandmark a addr:Landmark; addr:isLandmarkType ltype:Thoroughfare.
                    ?addrSeg a addr:AddressSegment; addr:relatum ?tmpLandmark; addr:nextStep [a addr:AddressSegment; addr:relatum ?tmpDistrict].
                    ?tmpDistrict a addr:Landmark; addr:isLandmarkType ltype:District.
                    ?district skos:exactMatch ?tmpDistrict.
                }}
                GRAPH ?gTmp {{
                    ?tmpLandmark skos:hiddenLabel ?normLabel.
                }}
            }}
        }}
        BIND(URI(CONCAT(STR(URI(ban:)), "LM_", STRUUID())) AS ?landmark)

        GRAPH ?g {{
            ?tmpLandmark a addr:Landmark; addr:isLandmarkType ltype:Thoroughfare.
                    ?addrSeg a addr:AddressSegment; addr:relatum ?tmpLandmark; addr:nextStep [a addr:AddressSegment; addr:relatum ?tmpDistrict].
                    ?tmpDistrict a addr:Landmark; addr:isLandmarkType ltype:District.
                    ?district skos:exactMatch ?tmpDistrict.
        }}
        GRAPH ?gTmp {{
            ?tmpLandmark skos:hiddenLabel ?normLabel.
        }}
    }}
    """

    # Transfert des données des ressources temporaires vers les permanentes.

    query5 = prefixes + f"""
    DELETE {{
        GRAPH ?g {{
            ?s ?p ?tmpLandmark.
            ?tmpLandmark ?p ?o.
        }}
    }}
    INSERT {{
        GRAPH ?g {{
            ?s ?p ?landmark.
            ?landmark ?p ?o.
        }}
    }}
    WHERE {{
        ?landmark skos:exactMatch ?tmpLandmark.
        GRAPH ?g {{
            {{?tmpLandmark ?p ?o}} UNION {{?s ?p ?tmpLandmark}}
          }}
    }} ; 

    DELETE {{
        ?landmark skos:exactMatch ?tmpLandmark.
    }}
    WHERE {{
        BIND({source_graph_uri.n3()} AS ?g)
        GRAPH ?g {{
            ?landmark skos:exactMatch ?tmpLandmark.
        }}
    }}
    """

    queries = [query1, query2, query3, query4, query5]
    for query in queries:
        gd.update_query(query, graphdb_url, project_name)

### Ajout d'événéments / de changements ainsi que des attributs à tous les repères

In [ ]:
def add_missing_elements_for_landmarks(graphdb_url, project_name, source_graph_uri):
    """
    Ajouter des éléments comme les changements, les événements, les attributs et leurs versions
    """

    query = f"""
    PREFIX geo: <http://www.opengis.net/ont/geosparql#>
    PREFIX geofla: <http://data.ign.fr/def/geofla#>
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
    PREFIX ban: <http://rdf.geohistoricaldata.org/id/address/sources/ban/>
    PREFIX ctype: <http://rdf.geohistoricaldata.org/id/codes/address/changeType/>
    PREFIX atype: <http://rdf.geohistoricaldata.org/id/codes/address/attributeType/>
    DELETE {{
        GRAPH ?g {{ 
            ?landmark geo:asWKT ?geom; geofla:numInsee ?inseeCode.
        }}
    }}
    INSERT {{
        GRAPH ?g {{
            ?landmark addr:hasAttribute ?nameAttribute, ?geomAttribute.
            ?nameAttribute a addr:Attribute; addr:isAttributeType atype:NameAttribute; addr:version ?versionNameAttribute1, ?versionNameAttribute2.
            ?geomAttribute a addr:Attribute; addr:isAttributeType atype:GeometryAttribute; addr:version ?versionGeomAttribute.
            ?versionNameAttribute1 a addr:AttributeVersion; addr:versionValue ?label.
            ?versionNameAttribute2 a addr:AttributeVersion; addr:versionValue ?inseeCode.
            ?versionGeomAttribute a addr:AttributeVersion; addr:versionValue ?geom.
        }}  
    }}
    WHERE {{
        BIND({source_graph_uri.n3()} AS ?g)
        GRAPH ?g {{
            {{
                SELECT * {{
                    ?landmark a addr:Landmark; rdfs:label ?label.
                    OPTIONAL {{?landmark geo:asWKT ?geom}}
                    OPTIONAL {{?landmark geofla:numInsee ?inseeCode}}
                }}
            }}
            BIND(URI(CONCAT(STR(URI(ban:)), "AN_", STRUUID())) AS ?nameAttribute)
            BIND(URI(CONCAT(STR(URI(ban:)), "ANV_", STRUUID())) AS ?versionNameAttribute1)
            BIND(IF(BOUND(?inseeCode), URI(CONCAT(STR(URI(ban:)), "ANV_", STRUUID())), ?x) AS ?versionNameAttribute2)
            BIND(IF(BOUND(?geom), URI(CONCAT(STR(URI(ban:)), "AG_", STRUUID())), ?x) AS ?geomAttribute)
            BIND(IF(BOUND(?geom), URI(CONCAT(STR(URI(ban:)), "AGV_", STRUUID())), ?x) AS ?versionGeomAttribute)
        }}
    }}
    """

    gd.update_query(query, graphdb_url, project_name)

#### Définition d'un processus de création des données de la BAN

In [ ]:
def create_factoid_process_ban(graphdb_url, ontorefine_url, ontorefine_cmd, project_name, ban_graph_name, facts_graph_name,
                               ban_csv_file, ban_mapping_file, ban_kg_file):
    """
    Fonction pour faire l'ensemble des processus relatifs à la création des factoïdes pour les données de la BAN
    """

    # Récupération des URI des graphes nommés
    ban_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, project_name, ban_graph_name))
    facts_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, project_name, facts_graph_name))

    # A partir des fichiers csv décrivant les adresses de la BAN dans Paris, convertir en un graphe de connaissance selon le mapping défini
    # Puis import du graphe dans le répertoire dont le nom est `project_name` et dans le graphe nommé donné par `graph_name`
    from_raw_to_data_to_graphdb(graphdb_url, ontorefine_url, ontorefine_cmd, project_name, ban_graph_name, ban_csv_file, ban_mapping_file, ban_kg_file)

    # Nettoyer les données en fusionnant les doublons après l'import dans GraphDB
    clean_ban_graph(graphdb_url, project_name, ban_graph_uri)

    # Ajout d'éléments manquants
    add_missing_elements_for_landmarks(graphdb_url, project_name, ban_graph_uri)

    # L'URI ci-dessous définit la source liée à la BAN
    ban_source_uri = URIRef("http://rdf.geohistoricaldata.org/id/address/facts/Source_BAN")
    create_source_ban(graphdb_url, project_name, ban_source_uri, facts_graph_uri)

    # Ajout de labels normalisés
    add_normalized_label_for_landmarks(graphdb_url, project_name, ban_graph_uri)
    
    # Ajout de liens entre les ressources de type repère et la source
    add_source_resources_links(graphdb_url, project_name, ban_source_uri, ban_graph_uri)

### Voies de Paris via Wikidata

#### Création des données liées à la source

In [ ]:
def create_source_wikidata(graphdb_url, project_name, source_resource_uri:URIRef, facts_graph_uri:URIRef):
    query = f"""
        PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
        PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
        PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>

        INSERT DATA {{
            GRAPH {facts_graph_uri.n3()} {{
                {source_resource_uri.n3()} a rico:Record;
                    rdfs:label "Wikidata"@fr.
            }}
        }}
        """
    
    gd.update_query(query, graphdb_url, project_name)

#### Sélection des voies de Paris sur Wikidata via une requête de sélection

Via une requête SPARQL de sélection, on récupère une liste de ressources à créér :
* pour chaque ressource décrivant une voie dans Paris, on créé autant de ressources qu'elle a de nom officiel. La place de la Nation ([wd:Q1573359](https://www.wikidata.org/entity/Q1573359)) a quatre noms officiels donc on aura 4 ressources. Pour celles qui n'ont pas de nom officiel, on créé une seule ressource dont le nom est le label en français
* ces ressources créées seront liées à celle de Wikidata dont elle provient via `skos:closeMatch`
* chaque ressource créée suit la structure définie par l'ontologie
* le résultat de la requête est stocké dans un fichier csv qui sera converti en graphe de connaissance via Ontotext Refine et un fichier de mapping 

In [ ]:
def get_paris_thoroughfares_from_wikidata(out_csv_file):
    query = f"""
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
    PREFIX source: <http://rdf.geohistoricaldata.org/id/address/sources/wikidata/> 
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
    PREFIX ps: <http://www.wikidata.org/prop/statement/>
    PREFIX p: <http://www.wikidata.org/prop/>
    PREFIX pqv: <http://www.wikidata.org/prop/qualifier/value/>
    PREFIX wb: <http://wikiba.se/ontology#>
    PREFIX time: <http://www.w3.org/2006/time#>

    SELECT ?landmarkId ?voie ?nomOff ?dateStartStamp ?dateStartPrec ?dateStartCal ?dateEndStamp ?dateEndPrec ?dateEndCal
    WHERE {{

        BIND(CONCAT("LM_", STRUUID()) AS ?landmarkId)
        {{
            SELECT DISTINCT * WHERE {{
                {{?voie p:P361 [ps:P361 wd:Q16024163].}}UNION{{?voie p:P361 [ps:P361 wd:Q107311481].}}
                {{?voie p:P1448 ?nomOffSt.
                    ?nomOffSt ps:P1448 ?nomOff.
                    OPTIONAL {{?nomOffSt pq:P580 ?dateStartStamp; pqv:P580 [wb:timePrecision ?dateStartPrecRaw; wb:timeCalendarModel ?dateStartCal]}}
                    OPTIONAL {{?nomOffSt pq:P582 ?dateEndStamp; pqv:P582 [wb:timePrecision ?dateEndPrecRaw; wb:timeCalendarModel ?dateEndCal]}}
                }}UNION{{
                    ?voie rdfs:label ?nomOff.
                    FILTER (LANG(?nomOff) = "fr")
                    MINUS {{?voie p:P1448 ?nomOffSt}}
                }}
            }}
        }}
        
        BIND(IF(?dateStartPrecRaw = 11, time:unitDay, 
                        IF(?dateStartPrecRaw = 10, time:unitMonth,
                            IF(?dateStartPrecRaw = 9, time:unitYear,
                                IF(?dateStartPrecRaw = 8, time:unitDecade,
                                    IF(?dateStartPrecRaw = 7, time:unitCentury,
                                        IF(?dateStartPrecRaw = 6, time:unitMillenium, ?x
                                    )))))) AS ?dateStartPrec)
        BIND(IF(?dateEndPrecRaw = 11, time:unitDay, 
                        IF(?dateEndPrecRaw = 10, time:unitMonth,
                            IF(?dateEndPrecRaw = 9, time:unitYear,
                                IF(?dateEndPrecRaw = 8, time:unitDecade,
                                    IF(?dateEndPrecRaw = 7, time:unitCentury,
                                        IF(?dateEndPrecRaw = 6, time:unitMillenium, ?x
                                    )))))) AS ?dateEndPrec)
    }}
    """

    wd.save_select_query_as_csv_file(query, out_csv_file)

#### Définition d'un processus de création des données de Wikidata

In [ ]:
def create_factoid_process_wikidata(graphdb_url, ontorefine_url, ontorefine_cmd, project_name, wdpt_graph_name, facts_graph_name,
                               wdpt_csv_file, wdpt_mapping_file, wdpt_kg_file):
    """
    Fonction pour faire l'ensemble des processus relatifs à la création des factoïdes pour les données de Wikidata
    """

    # Récupération des URI des graphes nommés
    wdpt_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, project_name, wdpt_graph_name))
    facts_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, project_name, facts_graph_name))

    # Récupération des données via le endpoint de Wikidatadans un fichier CSV
    # get_paris_thoroughfares_from_wikidata(wdpt_csv_file)

    # A partir des fichiers csv décrivant les adresses de la BAN dans Paris, convertir en un graphe de connaissance selon le mapping défini
    # Puis import du graphe dans le répertoire dont le nom est `project_name` et dans le graphe nommé donné par `graph_name`
    from_raw_to_data_to_graphdb(graphdb_url, ontorefine_url, ontorefine_cmd, project_name, wdpt_graph_name, wdpt_csv_file, wdpt_mapping_file, wdpt_kg_file)

    # Ajout d'éléments manquants
    add_missing_elements_for_landmarks(graphdb_url, project_name, wdpt_graph_uri)

    # L'URI ci-dessous définit la source liée à Wikidata
    wdpt_source_uri = URIRef("http://rdf.geohistoricaldata.org/id/address/facts/Source_WD")
    create_source_wikidata(graphdb_url, project_name, wdpt_source_uri, facts_graph_uri)

    # Ajout de labels normalisés
    add_normalized_label_for_landmarks(graphdb_url, project_name, wdpt_graph_uri)
    
    # Ajout de liens entre les ressources de type repère et la source
    add_source_resources_links(graphdb_url, project_name, wdpt_source_uri, wdpt_graph_uri)

# Création des faits

## Création de données dans le graphe des faits

### Définition de fonctions pour peupler le graphe des faits

In [ ]:
def create_unlinked_resources(graphdb_url, project_name, resource_class:str, resource_prefix:str, source_graph_uri:URIRef, facts_graph_uri:URIRef):
    # Créer des ressources en tant que fait et créer un lien de provenance
    query = f"""
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    INSERT {{
        GRAPH {source_graph_uri.n3()} {{
            ?resource owl:sameAs ?sourceResource.
        }}
        GRAPH {facts_graph_uri.n3()} {{
            ?resource a ?type.
        }}
    }}
    WHERE {{
        ?type rdfs:subClassOf* addr:{resource_class}.
        GRAPH {source_graph_uri.n3()} {{
            ?sourceResource a ?type.
        }}
        MINUS {{?sourceResource owl:sameAs [a addr:{resource_class}]}}
        BIND(URI(CONCAT(STR(URI(facts:)), "{resource_prefix}_", STRUUID())) AS ?resource)
    }}
    """

    gd.update_query(query, graphdb_url, project_name)

def create_linked_between_similar_thoroughfares(graphdb_url, project_name, source_graph_uri:URIRef, facts_graph_uri:URIRef):
    """
    Pour les repères de type VOIE définis dans le graphe nommé `source_graph_uri`, les lier avec un repère de même type défini dans `facts_graph_uri` s'ils ont un nom similaire.
    Le lien créé est mis dans `source_facts_graph_uri`.
    """
    prefixes = """
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    """

    query = prefixes + f"""
    INSERT {{
        GRAPH {source_graph_uri.n3()} {{
            ?factsLandmark owl:sameAs ?sourceLandmark.
        }}
    }}
    WHERE {{
        GRAPH {source_graph_uri.n3()} {{
            ?sourceLandmark a addr:Landmark; addr:isLandmarkType ltype:Thoroughfare.
        }}
        GRAPH {facts_graph_uri.n3()} {{
            ?factsLandmark a addr:Landmark.
           }}
        MINUS {{?factsLandmark owl:sameAs ?sourceLandmark}}
        ?sourceLandmark skos:hiddenLabel ?label.
        ?factsLandmark skos:hiddenLabel ?label.
    }}
    """

    gd.update_query(query, graphdb_url, project_name)

def select_transfer_implicit_triples(graphdb_url, project_name, tmp_folder, source_graph_uri:URIRef, facts_graph_uri:URIRef):
    query = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
    
    SELECT ?s ?p ?o FROM <http://www.ontotext.com/implicit>
        WHERE {{
            ?s ?p ?o.
            FILTER(?p in (addr:hasAttribute,addr:isAttributeType,addr:appliedTo,addr:dependsOn,addr:hasTime,addr:isChangeType,addr:isLandmarkType,addr:makesEffective,addr:outdates,addr:relatum,addr:timeCalendar,addr:timePrecision,addr:timeStamp,addr:versionValue,addr:version,rdfs:label,skos:hiddenLabel,rico:isOrWasDescribedBy))
            MINUS {{
                GRAPH {source_graph_uri.n3()} {{
                    ?s a ?typeS.
                }}
            }}
            MINUS {{
                GRAPH {source_graph_uri.n3()} {{
                    ?o a ?typeO.
                }}
            }}
            }}
    """

    res_query_file_name = "transfer_implicit_triples.csv"
    res_query_file = os.path.join(tmp_folder, res_query_file_name)

    gd.select_query_to_txt_file(query, graphdb_url, project_name, res_query_file)

def transfer_implicit_triples(graphdb_url, project_name, source_graph_uri:URIRef, facts_graph_uri:URIRef):
    query = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
    
    INSERT {{
        GRAPH {facts_graph_uri.n3()} {{
            ?s ?p ?o.
        }}
    }}
    WHERE {{
        GRAPH {facts_graph_uri.n3()} {{
                ?s a ?typeS.
        }}
        MINUS {{
            GRAPH {source_graph_uri.n3()} {{
                ?o a ?typeO.
            }} 
        }}
        ?s ?p ?o.

        MINUS {{
            GRAPH ?g {{
                ?s ?p ?o.
            }}
        }}
        FILTER(?p in (addr:hasAttribute,addr:isAttributeType,addr:appliedTo,addr:dependsOn,addr:hasTime,addr:isChangeType,addr:isLandmarkType,addr:makesEffective,addr:outdates,addr:relatum,addr:timeCalendar,addr:timePrecision,addr:timeStamp,addr:versionValue,addr:version,rdfs:label,skos:hiddenLabel,rico:isOrWasDescribedBy))
    }}
    """
    
    gd.update_query(query, graphdb_url, project_name)

### Création de voies à partir des différentes sources

La création de voies se fait de la manière suivante :
* on normalise les labels des repères, qui sont stockés dans un graphe nommé temporaire
* on sélectionne une source
* pour chaque ressource définie dans la source, on regarde si elle existe dans le graphe des faits (via des critères pré-définis comme le nom)
    * si la ressource existe, on crée un lien `?a addr:isReferencedBy ?b` décrivant le fait que la ressource existe
    * sinon, on la crée et on ajoute un lien `?a addr:isCreatedBy ?b`
* on refait le processus suivant pour une autre source jusqu'à les avoir toutes faites

In [ ]:
def insert_factoids_in_facts(graphdb_url, project_name, source_graph_uri:URIRef, facts_graph_uri:URIRef):
    create_linked_between_similar_thoroughfares(graphdb_url, project_name, source_graph_uri, facts_graph_uri)
    create_unlinked_resources(graphdb_url, project_name, "Landmark", "LM", source_graph_uri, facts_graph_uri)
    create_unlinked_resources(graphdb_url, project_name, "Address", "ADDR", source_graph_uri, facts_graph_uri)
    create_unlinked_resources(graphdb_url, project_name, "LandmarkRelation", "LR", source_graph_uri, facts_graph_uri)
    create_unlinked_resources(graphdb_url, project_name, "Attribute", "ATTR", source_graph_uri, facts_graph_uri)
    create_unlinked_resources(graphdb_url, project_name, "AttributeVersion", "AV", source_graph_uri, facts_graph_uri)
    create_unlinked_resources(graphdb_url, project_name, "Change", "CG", source_graph_uri, facts_graph_uri)
    create_unlinked_resources(graphdb_url, project_name, "Event", "EV", source_graph_uri, facts_graph_uri)
    create_unlinked_resources(graphdb_url, project_name, "TemporalEntity", "TE", source_graph_uri, facts_graph_uri)

## Processus final et itératif

In [ ]:
# Import des données de la ville de Paris
# create_factoid_process_ville_paris(graphdb_url, project_name, ontorefine_url, ontorefine_cmd, vpt_graph_name, facts_graph_name,
#                                    vpta_csv_file, vptc_csv_file, vpta_mapping_file, vptc_mapping_file, vpta_kg_file, vptc_kg_file)
# insert_factoids_in_facts(graphdb_url, project_name, vpt_graph_uri, facts_graph_uri)
# transfer_implicit_triples(graphdb_url, project_name, vpt_graph_uri, facts_graph_uri)
# gd.remove_graph(graphdb_url, project_name, vpt_graph_name)

# # Import des données de la BAN
# create_factoid_process_ban(graphdb_url, ontorefine_url, ontorefine_cmd, project_name, bpa_graph_name, facts_graph_name, bpa_csv_file, bpa_mapping_file, bpa_kg_file)
# insert_factoids_in_facts(graphdb_url, project_name, bpa_graph_uri, facts_graph_uri)
# transfer_implicit_triples(graphdb_url, project_name, bpa_graph_uri, facts_graph_uri)
# gd.remove_graph(graphdb_url, project_name, bpa_graph_name)

# # Import des données de Wikidata
# create_factoid_process_wikidata(graphdb_url, ontorefine_url, ontorefine_cmd, project_name, wdpt_graph_name, facts_graph_name, wdpt_csv_file, wdpt_mapping_file, wdpt_kg_file)
# insert_factoids_in_facts(graphdb_url, project_name, wdpt_graph_uri, facts_graph_uri)
# transfer_implicit_triples(graphdb_url, project_name, wdpt_graph_uri, facts_graph_uri)
# gd.remove_graph(graphdb_url, project_name, wdpt_graph_name)